In [1]:
import os

from utils.loaders import load_mnist

from models.VariationalAutoEncoder import VariationalAutoEncoder

Using TensorFlow backend.


## Set Run Parameters

In [2]:
# run params
SECTION = 'vae'
RUN_ID = '0001'
DATA_NAME = 'digits'
if not os.path.exists("run"):
    os.mkdir("run")
if not os.path.exists(f"run/{SECTION}"):
    os.mkdir(f"run/{SECTION}")
RUN_FOLDER = f'run/{SECTION}/'
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

MODE =  'build' #'load' #

## Load the data

In [3]:
(x_train, y_train), (x_test, y_test) = load_mnist()

## Define the structure of the neural network

In [4]:
encoder_architecture = [
    {'filter': 32, 'kernel': (3, 3), 'stride': 1},
    {'filter': 64, 'kernel': (3, 3), 'stride': 2},
    {'filter': 64, 'kernel': (3, 3), 'stride': 2},
    {'filter': 64, 'kernel': (3, 3), 'stride': 1},
    ]
decoder_architecture = [
    {'filter': 64, 'kernel': (3, 3), 'stride': 1},
    {'filter': 64, 'kernel': (3, 3), 'stride': 2},
    {'filter': 32, 'kernel': (3, 3), 'stride': 2},
    {'filter': 1, 'kernel': (3, 3), 'stride': 1},
    ]
input_dim = (28,28,1)
latent_dim = 2

In [5]:
ae = VariationalAutoEncoder(
    input_dim=input_dim,
    latent_dim=latent_dim,
    encoder_params=encoder_architecture,
    decoder_params=decoder_architecture
)

W0527 01:59:56.073784 139861816448768 deprecation.py:506] From /home/comadan/.venv/gdl/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [6]:
if MODE == 'build':
    ae.save(RUN_FOLDER)
else:
    ae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [7]:
ae.autoencoder_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 28, 28, 32)   320         encoder_input[0][0]              
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 28, 28, 32)   0           encoder_conv_0[0][0]             
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 14, 14, 64)   18496       leaky_re_lu_1[0][0]              
____________________________________________________________________________________________

In [8]:
ae.encoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 28, 28, 32)   320         encoder_input[0][0]              
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 28, 28, 32)   0           encoder_conv_0[0][0]             
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 14, 14, 64)   18496       leaky_re_lu_1[0][0]              
____________________________________________________________________________________________

In [9]:
ae.decoder_model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   (None, 2)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 3136)              9408      
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_0 (Conv2DTransp (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_1 (Conv2DTransp (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 14, 14, 64)        0   

## Train the autoencoder

In [10]:
LEARNING_RATE = 0.0005
BATCH_SIZE = 32
INITIAL_EPOCH = 0
TOTAL_EPOCHS = 200

In [11]:
ae.compile(LEARNING_RATE)

In [12]:
ae.train(     
    x_train, 
    batch_size = BATCH_SIZE, 
    epochs = TOTAL_EPOCHS, 
    run_folder = RUN_FOLDER, 
    initial_epoch = INITIAL_EPOCH
)

W0527 01:59:58.962708 139861816448768 module_wrapper.py:139] From /home/comadan/.venv/gdl/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/200
60000/60000 [==============================] - 19s 312us/step - loss: 58.3781 - reconstruction_loss: 55.0966 - kl_divergence_loss: 3.2814

Epoch 00001: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 2/200
60000/60000 [==============================] - 17s 277us/step - loss: 51.4880 - reconstruction_loss: 47.4430 - kl_divergence_loss: 4.0450

Epoch 00002: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 3/200
60000/60000 [==============================] - 17s 277us/step - loss: 49.9242 - reconstruction_loss: 45.5944 - kl_divergence_loss: 4.3298

Epoch 00003: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 4/200
60000/60000 [==============================] - 17s 279us/step - loss: 49.0310 - reconstruction_loss: 44.5366 - kl_divergence_loss: 4.4943

Epoch 00004: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 5/200
60000/60000 [==============================] - 17s 278us/step - loss: 48.3640 - reconstruction_loss: 43.7529 - k

60000/60000 [==============================] - 16s 272us/step - loss: 44.9427 - reconstruction_loss: 39.7714 - kl_divergence_loss: 5.1713

Epoch 00038: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 39/200
60000/60000 [==============================] - 16s 274us/step - loss: 44.8943 - reconstruction_loss: 39.7066 - kl_divergence_loss: 5.1878

Epoch 00039: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 40/200
60000/60000 [==============================] - 16s 274us/step - loss: 44.9117 - reconstruction_loss: 39.7309 - kl_divergence_loss: 5.1808

Epoch 00040: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 41/200
60000/60000 [==============================] - 16s 273us/step - loss: 44.8762 - reconstruction_loss: 39.6843 - kl_divergence_loss: 5.1919

Epoch 00041: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 42/200
60000/60000 [==============================] - 16s 273us/step - loss: 44.7797 - reconstruction_loss: 39.5950 - kl_diverg

60000/60000 [==============================] - 17s 283us/step - loss: 44.1939 - reconstruction_loss: 38.9039 - kl_divergence_loss: 5.2899

Epoch 00075: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 76/200
60000/60000 [==============================] - 17s 281us/step - loss: 44.1910 - reconstruction_loss: 38.9019 - kl_divergence_loss: 5.2892

Epoch 00076: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 77/200
60000/60000 [==============================] - 17s 281us/step - loss: 44.1680 - reconstruction_loss: 38.8648 - kl_divergence_loss: 5.3032

Epoch 00077: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 78/200
60000/60000 [==============================] - 17s 279us/step - loss: 44.1673 - reconstruction_loss: 38.8867 - kl_divergence_loss: 5.2805

Epoch 00078: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 79/200
60000/60000 [==============================] - 17s 280us/step - loss: 44.1444 - reconstruction_loss: 38.8419 - kl_diverg

60000/60000 [==============================] - 17s 277us/step - loss: 43.7794 - reconstruction_loss: 38.4108 - kl_divergence_loss: 5.3685

Epoch 00112: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 113/200
60000/60000 [==============================] - 17s 276us/step - loss: 43.7703 - reconstruction_loss: 38.4040 - kl_divergence_loss: 5.3662

Epoch 00113: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 114/200
60000/60000 [==============================] - 17s 278us/step - loss: 43.7595 - reconstruction_loss: 38.4005 - kl_divergence_loss: 5.3589

Epoch 00114: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 115/200
60000/60000 [==============================] - 17s 278us/step - loss: 43.7759 - reconstruction_loss: 38.3972 - kl_divergence_loss: 5.3787

Epoch 00115: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 116/200
60000/60000 [==============================] - 17s 280us/step - loss: 43.7328 - reconstruction_loss: 38.3565 - kl_di

60000/60000 [==============================] - 16s 275us/step - loss: 43.4973 - reconstruction_loss: 38.0800 - kl_divergence_loss: 5.4173

Epoch 00149: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 150/200
60000/60000 [==============================] - 17s 275us/step - loss: 43.4620 - reconstruction_loss: 38.0450 - kl_divergence_loss: 5.4171

Epoch 00150: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 151/200
60000/60000 [==============================] - 17s 275us/step - loss: 43.4660 - reconstruction_loss: 38.0444 - kl_divergence_loss: 5.4216

Epoch 00151: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 152/200
60000/60000 [==============================] - 17s 275us/step - loss: 43.4957 - reconstruction_loss: 38.0724 - kl_divergence_loss: 5.4233

Epoch 00152: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 153/200
60000/60000 [==============================] - 16s 274us/step - loss: 43.4793 - reconstruction_loss: 38.0527 - kl_di

60000/60000 [==============================] - 17s 275us/step - loss: 43.2542 - reconstruction_loss: 37.7931 - kl_divergence_loss: 5.4611

Epoch 00186: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 187/200
60000/60000 [==============================] - 17s 276us/step - loss: 43.2540 - reconstruction_loss: 37.7922 - kl_divergence_loss: 5.4618

Epoch 00187: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 188/200
60000/60000 [==============================] - 16s 275us/step - loss: 43.2904 - reconstruction_loss: 37.8302 - kl_divergence_loss: 5.4603

Epoch 00188: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 189/200
60000/60000 [==============================] - 17s 275us/step - loss: 43.2497 - reconstruction_loss: 37.7815 - kl_divergence_loss: 5.4682

Epoch 00189: saving model to run/vae/0001_digits/weights/weights.h5
Epoch 190/200
60000/60000 [==============================] - 17s 276us/step - loss: 43.2411 - reconstruction_loss: 37.7733 - kl_di